# 코테 준비 1주차

# Q1

![image](https://user-images.githubusercontent.com/61496071/202892074-a65a6ffc-f647-4c51-9248-9b151fb4b96a.png)

![image](https://user-images.githubusercontent.com/61496071/202892088-6950ada1-7f64-4bb0-b0f8-4c335ded5004.png)


In [ ]:
'''
1st try: Counter로 계산후 most_common() 사용 -> fail: 11, 18~20
'''

from collections import Counter

N,M = map(int, input().split())

# count
c = Counter()
for _ in range(M):
	events = input().split()[1:]
	c.update(events)
# count가 최댓값과 같은 key들
most = c.most_common(1)[0]
answer  = [k for k, v in c.most_common() if v == most[1]]

# 정렬
answer = reversed(answer)
print(' '.join(list(answer)))

'''
2nd try: fail : 6, 8
'''
from collections import Counter

N,M = map(int, input().split())

# count
c = Counter()
for _ in range(M):
	events = input().split()[1:]
	c.update(events)
"""# count가 최댓값과 같은 key들
most = c.most_common(1)[0]
answer  = [k for k, v in c.most_common() if v == most[1]]

# 정렬
answer = reversed(answer)"""
counts = c.most_common()
counts_sorted = sorted(counts, key=lambda x:(x[1], x[0]), reverse=True)
for k,v in counts_sorted:
	if v == counts_sorted[0][1]:
		print(k, end=' ')


# Q2

![image](https://user-images.githubusercontent.com/61496071/202892023-9c39302a-181e-46bf-a01e-284eab5e9bb0.png)

![image](https://user-images.githubusercontent.com/61496071/202892036-cde8bf6d-5264-49a3-b489-6831e2525c3b.png)


In [ ]:
'''
1st_try: 5/6/7 -> fail
union-find algorithm: 부모 노드에 따라 update 되는 알고리즘 시도
'''
from collections import defaultdict
N = int(input()) # 친구 수
M = int(input()) # 관계 수
links = defaultdict(list) 
parents = {i+1: i+1 for i in range(N)} # 처음엔 자기 자신이 부모로 init

def find(node):
	#node(idx)의 root idx를 return
	if parents[node] != node:
		return find(parents[node])
	return node
	
def union(x, y):
	#두 set를 합치고 reset
	x_root = find(x)
	y_root = find(y)
	if x_root < y_root:
		parents[y] = x_root # y node의 root를 x_root로 변경
		# y node의 children node들의 root 도 x_root로 변경
		for link in links[y]:
			if find(link) > x_root:
				parents[link] = x_root
			
	else:
		parents[x] = y_root
		for link in links[x]:
			if find(link) > y_root:
				parents[link] = y_root
	
for _ in range(M):
	u,v = map(int, input().split())
	links[u].append(v)
	links[v].append(u)
	union(u, v)

for parent in parents:
	parents[parent] = find(parent)
	
answer = [k for k, v in parents.items() if v == 1]
print(len(answer))


In [ ]:
7
5
1 4
3 5
7 6
1 5
4 3

In [ ]:
4

In [ ]:
5
5
1 3
2 3
3 4
4 5
4 2

In [ ]:
5

# Q3

![image](https://user-images.githubusercontent.com/61496071/202891881-37e145cf-43af-491a-a373-8e5e5d2f8075.png)

![image](https://user-images.githubusercontent.com/61496071/202891892-a1b5a5ad-70f2-43d5-871f-d5a3990c8338.png)

![image](https://user-images.githubusercontent.com/61496071/202891900-ff84ddb1-d26a-4033-9ba7-881dc7f9ebc9.png)


In [ ]:
'''
1:20 남음
최소 경로: 새 node를 거칠 때 마다 count를 기록하여 마지막에 최소 count를 선택
'''
from collections import defaultdict
N, M, K = map(int, input().split())
links = defaultdict(list)
for _ in range(M):
	a, b = map(int, input().split())
	links[a].append(b) # a -> b 단방향
dist = [N] * (N+1) # min_distance to the idx node
dist[K] = 0
visited = [0] * (N+1)
def update_dist(curr):
	visited[curr] = 1
	for link in links[curr]:
		if dist[link] > dist[curr]+1:
			dist[link] = dist[curr]+1
		if link == K and dist[link] == 0:
			dist[link] = dist[curr]+1
		if visited[link] == 0:
			update_dist(link)
update_dist(K)

if dist[K]==0:
	print(-1)
else:
	print(dist[K])

In [ ]:
5 6 4
1 2
1 3
3 2
4 2
5 1
5 4

In [ ]:
-1

In [ ]:
5 6 4
1 2
1 4
3 1
4 2
4 5
5 3

In [ ]:
4

# Q4

![image](https://user-images.githubusercontent.com/61496071/202891927-06b93ab1-ea79-4e7b-9d5f-21552beacc96.png)

![image](https://user-images.githubusercontent.com/61496071/202891960-9ec336d5-e923-4038-af00-05127a4071ee.png)

<img width="382" alt="image" src="https://user-images.githubusercontent.com/61496071/202891968-f71dd2aa-a9a3-404b-a466-eb76a9a56b22.png">

![image](https://user-images.githubusercontent.com/61496071/202891937-e6470109-6b4a-46a2-85ab-82c55e3c0fb1.png)


In [ ]:
from collections import defaultdict
"""
dfs로 cost1, cost2 를 노드 이동시마다 업데이트 하기
"""

N, M = map(int, input().split())
als = list(map(int, input().split()))
links = defaultdict(list)
times = [[0]*N for _ in range(N)]
for _ in range(M):
    u, v, w = map(int, input().split())
    links[u].append(v)
    links[v].append(u)
    times[u][v] = w
    times[v][u] = w
visited = [0] * (N+1)
def route(curr, visited, time, cost):
    '''time: curr올 때까지 누적 시간, cost: 나머지'''
    visited = visited.copy()
    visited[curr] = 1
    for _next in links[curr]:
        if visited[_next] == 1:
            continue
        new_cost = _next % 

In [ ]:
4 5
3 2 1 5
1 2 5
2 4 6
2 3 3
1 3 13
4 3 7

In [ ]:
15

In [ ]:
3 2
5 6 3
1 2 2
2 3 3

In [ ]:
-1